In [360]:
import pandas as pd 
import ast
import numpy as np
from nltk import word_tokenize
import re
import pickle

In [178]:
credits = pd.read_csv('tmdb_5000_credits.csv')
movies = pd.read_csv('tmdb_5000_movies.csv')

In [245]:
credits.info() 
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   title   4803 non-null   object
 1   cast    4803 non-null   object
 2   crew    4803 non-null   object
dtypes: object(3)
memory usage: 112.7+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        4803 non-null   int64 
 1   genres    4803 non-null   object
 2   overview  4800 non-null   object
 3   keywords  4803 non-null   object
 4   title     4803 non-null   object
dtypes: int64(1), object(4)
memory usage: 187.7+ KB


In [246]:
# removing unwanted features from the datasets and keeping only features which are usefuls 
movies = movies[['id', 'genres', 'overview', 'keywords', 'title' ]] 
credits = credits[['title', 'cast', 'crew']]                          

In [247]:
# Merging both the dataset based on common column  title
movies_data = pd.merge(movies, credits, on='title')
movies_data

,id,genres,overview,keywords,title,cast,crew
0,19995,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","In the 22nd century, a paraplegic Marine is di...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...","Captain Barbossa, long believed to be dead, ha...","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",A cryptic message from Bond’s past sends him o...,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",Following the death of District Attorney Harve...,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","John Carter is a war-weary, former military ca...","[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."
...,...,...,...,...,...,...,...
4804,9367,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",El Mariachi just wants to play his guitar and ...,"[{""id"": 5616, ""name"": ""united states\u2013mexi...",El Mariachi,"[{""cast_id"": 1, ""character"": ""El Mariachi"", ""c...","[{""credit_id"": ""52fe44eec3a36847f80b280b"", ""de..."
4805,72766,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 10749, ""...",A newlywed couple's honeymoon is upended by th...,[],Newlyweds,"[{""cast_id"": 1, ""character"": ""Buzzy"", ""credit_...","[{""credit_id"": ""52fe487dc3a368484e0fb013"", ""de..."
4806,231617,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 18, ""nam...","""Signed, Sealed, Delivered"" introduces a dedic...","[{""id"": 248, ""name"": ""date""}, {""id"": 699, ""nam...","Signed, Sealed, Delivered","[{""cast_id"": 8, ""character"": ""Oliver O\u2019To...","[{""credit_id"": ""52fe4df3c3a36847f8275ecf"", ""de..."
4807,126186,[],When ambitious New York attorney Sam is sent t...,[],Shanghai Calling,"[{""cast_id"": 3, ""character"": ""Sam"", ""credit_id...","[{""credit_id"": ""52fe4ad9c3a368484e16a36b"", ""de..."


In [248]:
# Dropping rows with missing values 
movies_data = movies_data.dropna()
movies_data.isna().sum()

id          0
genres      0
overview    0
keywords    0
title       0
cast        0
crew        0
dtype: int64

In [249]:
# converting from string representation to list of dicts and extracting the name tag from it and appending to list  
movies_data['genres'] = movies_data['genres'].apply(lambda x:[genre['name'] for genre in ast.literal_eval(x)])
movies_data['keywords'] = movies_data['keywords'].apply(lambda x:[keyword['name'] for keyword in ast.literal_eval(x)])

<ipython-input-249-8e472f402f3e>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies_data['genres'] = movies_data['genres'].apply(lambda x:[genre['name'] for genre in ast.literal_eval(x)])
<ipython-input-249-8e472f402f3e>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies_data['keywords'] = movies_data['keywords'].apply(lambda x:[keyword['name'] for keyword in ast.literal_eval(x)])


In [250]:
# Extracting only first 5 cast and first 2 crew names of each movies 
movies_data['cast'] = movies_data['cast'].apply(lambda x:[cast['name'] for cast in ast.literal_eval(x)[0:5]])
movies_data['crew'] = movies_data['crew'].apply(lambda x:[crew['name'] for crew in ast.literal_eval(x)[0:2]])

<ipython-input-250-c32308f281be>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies_data['cast'] = movies_data['cast'].apply(lambda x:[cast['name'] for cast in ast.literal_eval(x)[0:5]])
<ipython-input-250-c32308f281be>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies_data['crew'] = movies_data['crew'].apply(lambda x:[crew['name'] for crew in ast.literal_eval(x)[0:2]])


In [251]:
# Getting list of words from the overview columns 
movies_data['overview'] = movies_data['overview'].apply(word_tokenize)

<ipython-input-251-e2780c57f651>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies_data['overview'] = movies_data['overview'].apply(word_tokenize)


In [252]:
# removing duplicate values 
movies_data['crew'] = movies_data['crew'].apply(lambda x: list(dict.fromkeys(x)))

<ipython-input-252-b4cb53590830>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies_data['crew'] = movies_data['crew'].apply(lambda x: list(dict.fromkeys(x)))


In [253]:
# creating tag column from other columns 
movies_data['tags'] = movies_data['genres'] + movies_data['overview'] + movies_data['keywords'] + movies_data['cast'] + movies_data['crew']

<ipython-input-253-20cc7b181da6>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies_data['tags'] = movies_data['genres'] + movies_data['overview'] + movies_data['keywords'] + movies_data['cast'] + movies_data['crew']


In [254]:
movies_data.drop(['genres', 'overview','keywords', 'cast', 'crew'], axis=1)

,id,title,tags
0,19995,Avatar,"[Action, Adventure, Fantasy, Science Fiction, ..."
1,285,Pirates of the Caribbean: At World's End,"[Adventure, Fantasy, Action, Captain, Barbossa..."
2,206647,Spectre,"[Action, Adventure, Crime, A, cryptic, message..."
3,49026,The Dark Knight Rises,"[Action, Crime, Drama, Thriller, Following, th..."
4,49529,John Carter,"[Action, Adventure, Science Fiction, John, Car..."
...,...,...,...
4804,9367,El Mariachi,"[Action, Crime, Thriller, El, Mariachi, just, ..."
4805,72766,Newlyweds,"[Comedy, Romance, A, newlywed, couple, 's, hon..."
4806,231617,"Signed, Sealed, Delivered","[Comedy, Drama, Romance, TV Movie, ``, Signed,..."
4807,126186,Shanghai Calling,"[When, ambitious, New, York, attorney, Sam, is..."


In [255]:
# forming corpus from a list of words
movies_data['tags'] = movies_data['tags'].apply(lambda x: ["".join(word.split(" ")) for word in x])

<ipython-input-255-d9dd4d88e06e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies_data['tags'] = movies_data['tags'].apply(lambda x: ["".join(word.split(" ")) for word in x])


In [256]:
movies_data['tags'] = movies_data['tags'].apply(lambda x: [word for word in x if re.search('^[A-Za-z0-9_]+$', word)])

<ipython-input-256-03e65e779e12>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies_data['tags'] = movies_data['tags'].apply(lambda x: [word for word in x if re.search('^[A-Za-z0-9_]+$', word)])


In [258]:
movies_data['tags'] = movies_data['tags'].apply(lambda x: " ".join(x).casefold())

<ipython-input-258-791c9ce8a86b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies_data['tags'] = movies_data['tags'].apply(lambda x: " ".join(x).casefold())


In [ ]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [271]:
def stem(text):
    stem_words = [ps.stem(word) for word in text.split()]
    stem_tags = " ".join(stem_words)
    return stem_tags

In [272]:
movies_data['tags'] = movies_data['tags'].apply(stem)

<ipython-input-272-1571b14b8aa2>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies_data['tags'] = movies_data['tags'].apply(stem)


In [273]:
from sklearn.feature_extraction.text import CountVectorizer

In [277]:
cv = CountVectorizer(stop_words = 'english', max_features=5000)
vector = cv.fit_transform(movies_data['tags']).toarray()

In [285]:
vector.shape

(4806, 5000)

In [282]:
from sklearn.metrics.pairwise import cosine_similarity

(4806, 4806)

In [287]:
similarity = cosine_similarity(vector)

In [323]:
sorted(list(enumerate(similarity[0])), reverse = True, key=lambda x:x[1])

[(0, 1.0000000000000002),
 (507, 0.25016291953504993),
 (1216, 0.24364057665669586),
 (539, 0.2369241569216954),
 (2409, 0.2360960823249428),
 (1204, 0.2335709179335258),
 (61, 0.22417941532712204),
 (778, 0.22205779584216376),
 (1194, 0.21982600255746473),
 (1920, 0.21726204731337043),
 (260, 0.21653278478430668),
 (2999, 0.21483446221182986),
 (2786, 0.21182963643408081),
 (91, 0.21059925059706258),
 (973, 0.20814536170751838),
 (83, 0.20380986614602725),
 (1089, 0.2020475485519274),
 (577, 0.20131905799006777),
 (1831, 0.19795764159656265),
 (2333, 0.19795764159656265),
 (322, 0.1976664844429603),
 (4192, 0.19611613513818407),
 (74, 0.19611613513818402),
 (582, 0.19611613513818402),
 (3608, 0.19562767576168477),
 (2204, 0.19535451620220173),
 (1444, 0.19418390934515434),
 (3730, 0.19418390934515434),
 (301, 0.1933398313652378),
 (4048, 0.19277164398874672),
 (972, 0.19223226273338134),
 (466, 0.1921537845661046),
 (172, 0.19215378456610457),
 (1329, 0.19215378456610455),
 (300, 0.18

In [369]:
def recommend(title):
    
    movie_index = movies_data[movies_data['title']== title].index[0]
    recommended_object = sorted(list(enumerate(similarity[movie_index])), reverse = True, key=lambda x:x[1])[0:5]
    
    recommended_indexes = [i[0] for i in recommended_object]
    movie_title = list(movies_data.loc[recommended_indexes, 'title'])
    movie_id = list(movies_data.loc[recommended_indexes, 'id'])
    list_of_tuples = list(zip(movie_id, movie_title))
    
    return list_of_tuples

    

In [361]:
pickle.dump(movies_data.to_dict(), open('movie_dict.pkl', 'wb'))

In [362]:
pickle.dump(similarity, open('similarity.pkl', 'wb'))

In [372]:
for movie in recommend('Batman Begins'):
    print(movie[0])

272
155
268
268
49026
